In [1]:
import pandas as pd

In [3]:
job_nyc = pd.read_csv(r"C:\Users\Sina A.Y\Documents\Ironhack\first_project\notebooks\Jobs_NYC_Postings.csv")


In [5]:
job_nyc1 = pd.read_csv(r"C:\Users\Sina A.Y\Documents\Ironhack\first_project\notebooks\Jobs_NYC_Postings2.csv")


In [9]:
print(job_nyc1)


      Job ID                          Agency Posting Type  # Of Positions  \
0     534519  DEPT OF ENVIRONMENT PROTECTION     Internal               2   
1     631363  DEPT OF ENVIRONMENT PROTECTION     Internal               1   
2     628158         BRONX DISTRICT ATTORNEY     External               2   
3     629002      OFFICE OF CRIMINAL JUSTICE     Internal               1   
4     613253   DEPT OF DESIGN & CONSTRUCTION     Internal               1   
...      ...                             ...          ...             ...   
5064  609088  DEPT OF ENVIRONMENT PROTECTION     Internal               1   
5065  632969    DEPARTMENT OF TRANSPORTATION     Internal               3   
5066  508258        DEPARTMENT OF CORRECTION     Internal               1   
5067  543713           NYC HOUSING AUTHORITY     Internal               3   
5068  634000               POLICE DEPARTMENT     Internal               1   

                                         Business Title  \
0               

In [12]:
job_nyc["Preferred Skills"].isnull().sum()


np.int64(2699)

In [13]:
job_nyc["Preferred Skills"].value_counts(dropna=False)

Preferred Skills
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     2699
Candidates should have the ability to manage construction managers; complete multiple multi-trade projects on schedule; possess strong computer, organizational, verbal, and written communication skills; and should be attentive to details.                                                                                            

In [14]:
job_nyc.isnull().sum()

Job ID                              0
Agency                              0
Posting Type                        0
# Of Positions                      0
Business Title                      0
Civil Service Title                 0
Title Classification                0
Title Code No                       0
Level                               0
Job Category                        0
Full-Time/Part-Time indicator       0
Career Level                        0
Salary Range From                   0
Salary Range To                     0
Salary Frequency                    0
Work Location                       0
Division/Work Unit                  0
Job Description                     0
Minimum Qual Requirements          67
Preferred Skills                 2699
Additional Information           3742
To Apply                         3018
Hours/Shift                      4854
Work Location 1                  4571
Recruitment Contact              6030
Residency Requirement               0
Posting Date

In [17]:
print(pd.DataFrame({
    "non_nulls": job_nyc.count(),
    "nulls": job_nyc.isnull().sum(),
    "total": len(job_nyc)
}))



                               non_nulls  nulls  total
Job ID                              6030      0   6030
Agency                              6030      0   6030
Posting Type                        6030      0   6030
# Of Positions                      6030      0   6030
Business Title                      6030      0   6030
Civil Service Title                 6030      0   6030
Title Classification                6030      0   6030
Title Code No                       6030      0   6030
Level                               6030      0   6030
Job Category                        6030      0   6030
Full-Time/Part-Time indicator       6030      0   6030
Career Level                        6030      0   6030
Salary Range From                   6030      0   6030
Salary Range To                     6030      0   6030
Salary Frequency                    6030      0   6030
Work Location                       6030      0   6030
Division/Work Unit                  6030      0   6030
Job Descri

In [18]:
print(pd.DataFrame({
    "non_nulls": job_nyc1.count(),
    "nulls": job_nyc1.isnull().sum(),
    "total": len(job_nyc1)
}))

                               non_nulls  nulls  total
Job ID                              5069      0   5069
Agency                              5069      0   5069
Posting Type                        5069      0   5069
# Of Positions                      5069      0   5069
Business Title                      5069      0   5069
Civil Service Title                 5069      0   5069
Title Classification                5069      0   5069
Title Code No                       5069      0   5069
Level                               5069      0   5069
Job Category                        5069      0   5069
Full-Time/Part-Time indicator       4990     79   5069
Career Level                        5069      0   5069
Salary Range From                   5069      0   5069
Salary Range To                     5069      0   5069
Salary Frequency                    5069      0   5069
Work Location                       5069      0   5069
Division/Work Unit                  5069      0   5069
Job Descri

In [20]:
merged = pd.concat([job_nyc,job_nyc1], ignore_index = True)

In [21]:
merged = merged.drop_duplicates()

In [22]:
print(merged)

       Job ID                          Agency Posting Type  # Of Positions  \
0      605239  DEPT OF ENVIRONMENT PROTECTION     External               1   
1      680227  DEPT OF ENVIRONMENT PROTECTION     External               1   
2      634027     HRA/DEPT OF SOCIAL SERVICES     External               2   
3      564043  DEPT OF ENVIRONMENT PROTECTION     External               1   
4      556627  DEPT OF ENVIRONMENT PROTECTION     Internal               1   
...       ...                             ...          ...             ...   
11094  609088  DEPT OF ENVIRONMENT PROTECTION     Internal               1   
11095  632969    DEPARTMENT OF TRANSPORTATION     Internal               3   
11096  508258        DEPARTMENT OF CORRECTION     Internal               1   
11097  543713           NYC HOUSING AUTHORITY     Internal               3   
11098  634000               POLICE DEPARTMENT     Internal               1   

                                Business Title  \
0       AREA 

In [24]:
merged.to_csv("jobdata.csv", index=False)

In [31]:
print(merged.columns)

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Classification', 'Title Code No', 'Level',
       'Job Category', 'Full-Time/Part-Time indicator', 'Career Level',
       'Salary Range From', 'Salary Range To', 'Salary Frequency',
       'Work Location', 'Division/Work Unit', 'Job Description',
       'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')


In [32]:
merged = merged.drop_duplicates(subset=['Job ID'])

In [33]:
merged.to_csv("jobdata_dropped.csv", index=False)